In [ ]:
!pip install -qU "google-genai>=1.7.0"

In [ ]:
from google import genai
from kaggle_secrets import UserSecretsClient
from google.genai import types
import json

user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

MODEL_ID = "models/gemini-2.0-flash-exp"

In [ ]:
# Prompt with grounded context
def build_few_shot_prompt(user_opinion):
    example = """
Example:
User opinion: "Social media is harmful to mental health."

{
  "strengthened_argument": "Studies show increased social media use correlates with anxiety and depression...",
  "counter_argument": "Some argue social media fosters support networks...",
  "rebuttal": "Support exists, but the design prioritizes engagement over mental well-being."
}
"""
    return f"""
You are an AI debate assistant.

Respond ONLY in JSON with:
- strengthened_argument
- counter_argument
- rebuttal

{example}

Now analyze:
User opinion: "{user_opinion}"
"""


user_opinion = "Climate change is real."
prompt = build_few_shot_prompt(user_opinion)

config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)

response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config=config_with_search,
)

In [ ]:
import json
import re

def extract_json_from_response(text):
    # Try to find a JSON code block first
    match = re.search(r"```json(.*?)```", text, re.DOTALL)
    if match:
        json_str = match.group(1).strip()
    else:
        # Fallback: find the first { ... } block
        match = re.search(r"{.*}", text, re.DOTALL)
        if match:
            json_str = match.group(0).strip()
        else:
            return None

    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        return None

parsed = extract_json_from_response(response.text)

if parsed:
    print("User opinion: ", user_opinion)
    print("📢 Strengthened Argument:\n", parsed["strengthened_argument"])
    print("\n⚔️ Counter-Argument:\n", parsed["counter_argument"])
    print("\n🛡️ Rebuttal:\n", parsed["rebuttal"])
else:
    print("⚠️ Still couldn't parse response. Raw output:")
    print(response.text)

In [ ]:
# Evaluate the quality of the generated arguments
evaluation_prompt = f"""You are a critical thinking expert.
Evaluate the following debate arguments on a scale of 1 to 5 for:
- Relevance
- Logic
- Persuasiveness

Then provide brief feedback.

Input:
{json.dumps(parsed, indent=2)}

Respond in JSON:
{{
  "strengthened_argument_score": ...,
  "counter_argument_score": ...,
  "rebuttal_score": ...,
  "feedback": "..."
}}
"""

eval_response = client.models.generate_content(
    model=MODEL_ID,
    contents=evaluation_prompt
)

In [ ]:
# Clean and display eval response
eval_text = eval_response.text.strip()
if eval_text.startswith("```json"):
    eval_text = re.sub(r"^```json\s*", "", eval_text)
    eval_text = re.sub(r"```\s*$", "", eval_text)

try:
    eval_data = json.loads(eval_text)
    print("🧪 Evaluation Scores:")
    print(json.dumps(eval_data, indent=2))
except Exception as e:
    print("⚠️ Could not parse evaluation response:")
    print(eval_response.text)